# Optimisation Algorithm





| Algorithm       | Learning Rate Adaptation | Momentum | Adaptive Learning Rate | Suitable for Large Networks | Suitable for Noisy Data | Convergence Speed | Memory Requirements |
|-----------------|-------------------------|----------|------------------------|----------------------------|------------------------|-------------------|---------------------|
| Stochastic Gradient Descent (SGD) | No          | Yes      | No                     | Yes                        | Yes                    | Slow              | Low                 |
| Adam            | Yes                     | Yes      | Yes                    | Yes                        | Yes                    | Fast              | Moderate            |
| RMSprop         | Yes                     | No       | Yes                    | Yes                        | Yes                    | Fast              | Moderate            |
| AdaDelta        | Yes                     | No       | Yes                    | Yes                        | Yes                    | Fast              | High                |
| Adamax          | Yes                     | Yes      | Yes                    | Yes                        | Yes                    | Fast              | Moderate            |
| Nadam           | Yes                     | Yes      | Yes                    | Yes                        | Yes                    | Fast              | Moderate            |
| AMSGrad         | Yes                     | Yes      | Yes                    | Yes                        | Yes                    | Fast              | Moderate            |
| L-BFGS          | No                      | No       | No                     | No                         | Yes                    | Fast              | High                |


### Stochastic Gradiant Descent (SGD):

Stochastic Gradient Descent (SGD): SGD is a widely used optimization algorithm for neural networks. It updates the model's parameters based on the gradients computed on mini-batches of training data. SGD is computationally efficient and can handle large datasets.

$$\text{Beta} \leftarrow \text{Beta} - \alpha \cdot \nabla J$$

In [ ]:
import numpy as np

def stochastic_gradient_descent_step(Beta, alpha, gradient_J):
    Beta -= alpha * gradient_J
    return Beta


### Adaptive Moment Estimation (Adam):

Adam: Adam (Adaptive Moment Estimation) is an adaptive optimization algorithm that combines ideas from both momentum and RMSprop. It adjusts the learning rate for each parameter based on the estimates of the first and second moments of the gradients.

$$m_i \leftarrow \beta_1 \cdot m_i + (1 - \beta_1) \cdot \nabla J \\
v_i \leftarrow \beta_2 \cdot v_i + (1 - \beta_2) \cdot (\nabla J)^2 \\
\hat{m} \leftarrow \frac{m_i}{1 - \beta_1} \\
\hat{v} \leftarrow \frac{v_i}{1 - \beta_2} \\
\text{param} \leftarrow \text{param} - \frac{\alpha \cdot \hat{m}}{\sqrt{\hat{v}} + \epsilon} \\
$$

In [1]:
import numpy as np

def adam_optimizer(parameters, gradients, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Adam optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - learning_rate: The learning rate (alpha) for the optimizer.
    - beta1: Exponential decay rate for the first moment estimates.
    - beta2: Exponential decay rate for the second moment estimates.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying Adam optimization.
    """

    # Initialize first and second moment estimates
    m = {}
    v = {}
    t = 0  # Time step

    for param_name, gradient in gradients.items():
        m[param_name] = np.zeros_like(gradient)
        v[param_name] = np.zeros_like(gradient)

    updated_parameters = {}

    for param_name, param_values in parameters.items():
        t += 1
        gradient = gradients[param_name]

        # Update first moment estimate
        m[param_name] = beta1 * m[param_name] + (1 - beta1) * gradient

        # Update second moment estimate
        v[param_name] = beta2 * v[param_name] + (1 - beta2) * gradient ** 2

        # Bias-corrected moment estimates
        m_hat = m[param_name] / (1 - beta1 ** t)
        v_hat = v[param_name] / (1 - beta2 ** t)

        # Update parameters
        updated_parameters[param_name] = param_values - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

# Apply Adam optimization
updated_parameters = adam_optimizer(initial_parameters, gradients, learning_rate, beta1, beta2, epsilon)
print(updated_parameters)


{'param1': 0.4990000001, 'param2': -0.7992558632115032}


### RMSprop

RMSprop: RMSprop (Root Mean Square Propagation) is an optimization algorithm that adapts the learning rate for each parameter based on the moving average of squared gradients. It helps in controlling the step size during training and can converge faster than basic SGD.

$$\text{cache}_i \leftarrow \text{decay\_rate} \cdot \text{cache}_i + (1 - \text{decay\_rate}) \cdot (\nabla J)^2 \\
\text{param} \leftarrow \text{param} - \frac{\alpha \cdot \nabla J}{\sqrt{\text{cache}_i} + \epsilon} \\
$$

In [2]:
import numpy as np

def rmsprop_optimizer(parameters, gradients, learning_rate, decay_rate=0.9, epsilon=1e-8):
    """
    RMSprop optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - learning_rate: The learning rate (alpha) for the optimizer.
    - decay_rate: Decay rate for the moving average of squared gradients.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying RMSprop optimization.
    """

    # Initialize cache for squared gradients
    cache = {}

    updated_parameters = {}

    for param_name, gradient in gradients.items():
        if param_name not in cache:
            cache[param_name] = np.zeros_like(gradient)

        # Update cache for squared gradients
        cache[param_name] = decay_rate * cache[param_name] + (1 - decay_rate) * gradient ** 2

        # Update parameters
        updated_parameters[param_name] = parameters[param_name] - learning_rate * gradient / (np.sqrt(cache[param_name]) + epsilon)

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
learning_rate = 0.001
decay_rate = 0.9
epsilon = 1e-8

# Apply RMSprop optimization
updated_parameters = rmsprop_optimizer(initial_parameters, gradients, learning_rate, decay_rate, epsilon)
print(updated_parameters)


{'param1': 0.4968377233398313, 'param2': -0.796837722673165}


### Adadelta

Adadelta: Adadelta is an extension of Adagrad that addresses its aggressive learning rate decay. It uses the accumulated gradients to adaptively adjust the learning rate, resulting in more stable convergence.

$$
\text{accum\_grad}_i \leftarrow \text{decay\_rate} \cdot \text{accum\_grad}_i + (1 - \text{decay\_rate}) \cdot (\nabla J)^2 \\
\delta \leftarrow -\frac{\sqrt{\text{accum\_delta}_i} + \epsilon}{\sqrt{\text{accum\_grad}_i} + \epsilon} \cdot \nabla J \\
\text{accum\_delta}_i \leftarrow \text{decay\_rate} \cdot \text{accum\_delta}_i + (1 - \text{decay\_rate}) \cdot \delta^2 \\
\text{param} \leftarrow \text{param} + \delta \\
$$

In [3]:
import numpy as np

def adadelta_optimizer(parameters, gradients, decay_rate=0.9, epsilon=1e-8):
    """
    Adadelta optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - decay_rate: Decay rate for the moving average of squared gradients and deltas.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying Adadelta optimization.
    """

    # Initialize accumulators for squared gradients and deltas
    accum_grad = {}
    accum_delta = {}

    updated_parameters = {}

    for param_name, gradient in gradients.items():
        if param_name not in accum_grad:
            accum_grad[param_name] = np.zeros_like(gradient)
            accum_delta[param_name] = np.zeros_like(gradient)

        # Update accumulator for squared gradients
        accum_grad[param_name] = decay_rate * accum_grad[param_name] + (1 - decay_rate) * gradient ** 2

        # Compute update step
        delta = -np.sqrt(accum_delta[param_name] + epsilon) / np.sqrt(accum_grad[param_name] + epsilon) * gradient

        # Update accumulator for squared deltas
        accum_delta[param_name] = decay_rate * accum_delta[param_name] + (1 - decay_rate) * delta ** 2

        # Update parameters
        updated_parameters[param_name] = parameters[param_name] + delta

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
decay_rate = 0.9
epsilon = 1e-8

# Apply Adadelta optimization
updated_parameters = adadelta_optimizer(initial_parameters, gradients, decay_rate, epsilon)
print(updated_parameters)


{'param1': 0.49968377381511014, 'param2': -0.7996837724096652}


### Adamax

Adamax: Adamax is a variant of Adam that uses the infinity norm (maximum absolute value) of the gradient instead of the L2 norm. It is more robust to sparse gradients and can be effective in large-scale applications.

$$m_i \leftarrow \beta_1 \cdot m_i + (1 - \beta_1) \cdot \nabla J \\
u_i \leftarrow \max(\beta_2 \cdot u_i, |\nabla J|) \\
\text{param} \leftarrow \text{param} - \frac{\alpha \cdot m_i}{u_i + \epsilon} \\
$$



In [4]:
import numpy as np

def adamax_optimizer(parameters, gradients, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Adamax optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - learning_rate: The learning rate (alpha) for the optimizer.
    - beta1: Exponential decay rate for the first moment estimates.
    - beta2: Exponential decay rate for the weighted infinity norm.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying Adamax optimization.
    """

    # Initialize first moment estimates and weighted infinity norm
    m = {}
    u = {}

    t = 0  # Time step

    for param_name, gradient in gradients.items():
        m[param_name] = np.zeros_like(gradient)
        u[param_name] = np.zeros_like(gradient)

    updated_parameters = {}

    for param_name, param_values in parameters.items():
        t += 1
        gradient = gradients[param_name]

        # Update first moment estimate
        m[param_name] = beta1 * m[param_name] + (1 - beta1) * gradient

        # Update weighted infinity norm
        u[param_name] = np.maximum(beta2 * u[param_name], np.abs(gradient))

        # Update parameters
        updated_parameters[param_name] = param_values - learning_rate * m[param_name] / (u[param_name] + epsilon)

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

# Apply Adamax optimization
updated_parameters = adamax_optimizer(initial_parameters, gradients, learning_rate, beta1, beta2, epsilon)
print(updated_parameters)

{'param1': 0.49990000001, 'param2': -0.7999000000033334}


### Nadam

Nadam: Nadam is a combination of Nesterov accelerated gradient (NAG) and Adam. It incorporates the benefits of NAG's momentum and Adam's adaptive learning rate to provide faster convergence and better generalization.

$$m_i \leftarrow \beta_1 \cdot m_i + (1 - \beta_1) \cdot \nabla J \\
v_i \leftarrow \beta_2 \cdot v_i + (1 - \beta_2) \cdot (\nabla J)^2 \\
\hat{m} \leftarrow \frac{m_i}{1 - \beta_1} \\
\hat{v} \leftarrow \frac{v_i}{1 - \beta_2} \\
\text{param} \leftarrow \text{param} - \frac{\alpha \cdot (\beta_1 \cdot \hat{m} + (1 - \beta_1) \cdot \nabla J)}{\sqrt{\hat{v}} + \epsilon} \\
$$

In [5]:
import numpy as np

def nadam_optimizer(parameters, gradients, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Nadam optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - learning_rate: The learning rate (alpha) for the optimizer.
    - beta1: Exponential decay rate for the first moment estimates.
    - beta2: Exponential decay rate for the second moment estimates.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying Nadam optimization.
    """

    # Initialize first and second moment estimates
    m = {}
    v = {}
    t = 0  # Time step

    for param_name, gradient in gradients.items():
        m[param_name] = np.zeros_like(gradient)
        v[param_name] = np.zeros_like(gradient)

    updated_parameters = {}

    for param_name, param_values in parameters.items():
        t += 1
        gradient = gradients[param_name]

        # Update first moment estimate
        m[param_name] = beta1 * m[param_name] + (1 - beta1) * gradient

        # Update second moment estimate
        v[param_name] = beta2 * v[param_name] + (1 - beta2) * gradient ** 2

        # Bias-corrected moment estimates
        m_hat = m[param_name] / (1 - beta1 ** t)
        v_hat = v[param_name] / (1 - beta2 ** t)

        # Update parameters
        updated_parameters[param_name] = param_values - learning_rate * ((beta1 * m_hat) + ((1 - beta1) * gradient)) / (np.sqrt(v_hat) + epsilon)

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

# Apply Nadam optimization
updated_parameters = nadam_optimizer(initial_parameters, gradients, learning_rate, beta1, beta2, epsilon)
print(updated_parameters)


{'param1': 0.4990000001, 'param2': -0.7991888909005386}


### AdaGrad-Delta

AdaGrad-Delta: AdaGrad-Delta is an extension of Adagrad that addresses its rapid learning rate decay. It introduces a decaying sum of squared gradients to adapt the learning rate and control the step size more effectively.

$$\text{cache}_i \leftarrow \text{cache}_i + (\nabla J)^2 \\
\delta_i \leftarrow -\frac{\sqrt{\delta_i} + \epsilon}{\sqrt{\text{cache}_i} + \epsilon} \cdot \nabla J \\
\text{param} \leftarrow \text{param} + \delta_i \\
$$

In [6]:
import numpy as np

def adagrad_delta_optimizer(parameters, gradients, learning_rate, epsilon=1e-8):
    """
    AdaGrad-Delta optimizer implementation.

    Parameters:
    - parameters: A dictionary containing the model parameters to be updated.
    - gradients: A dictionary containing the gradients of the parameters.
    - learning_rate: The learning rate (alpha) for the optimizer.
    - epsilon: Small value to prevent division by zero.

    Returns:
    - updated_parameters: The updated parameters after applying AdaGrad-Delta optimization.
    """

    # Initialize cache for squared gradients and deltas
    cache = {}
    delta = {}

    updated_parameters = {}

    for param_name, gradient in gradients.items():
        if param_name not in cache:
            cache[param_name] = np.zeros_like(gradient)
            delta[param_name] = np.zeros_like(gradient)

        # Update cache for squared gradients
        cache[param_name] += gradient ** 2

        # Compute update step
        delta[param_name] = -np.sqrt(delta[param_name] + epsilon) / np.sqrt(cache[param_name] + epsilon) * gradient

        # Update parameters
        updated_parameters[param_name] = parameters[param_name] + delta[param_name]

    return updated_parameters

# Example usage
# Initialize parameters and gradients
initial_parameters = {'param1': 0.5, 'param2': -0.8}
gradients = {'param1': 0.1, 'param2': -0.3}

# Set hyperparameters
learning_rate = 0.001
epsilon = 1e-8

# Apply AdaGrad-Delta optimization
updated_parameters = adagrad_delta_optimizer(initial_parameters, gradients, learning_rate, epsilon)
print(updated_parameters)


{'param1': 0.49990000004999996, 'param2': -0.7999000000055556}


### L-BFGS

L-BFGS: L-BFGS (Limited-memory Broyden-Fletcher-Goldfarb-Shanno) is a quasi-Newton optimization algorithm that approximates the Hessian matrix. It can be efficient for small-scale problems but may not scale well to large neural networks.

From an initial guess $\mathbf{x}_0$ and an approximate inverted Hessian matrix $H_0$, the following steps are repeated as $\mathbf{x}_k$ converges to the solution:

1. Obtain a direction $\mathbf{p}_k$ by solving $\mathbf{p}_k = -H_k \nabla J(\mathbf{x}_k)$, where $\nabla J(\mathbf{x}_k)$ is the gradient of the function at $\mathbf{x}_k$.
2. Perform a one-dimensional optimization (line search) to find an acceptable step size $\alpha_k$ in the direction found in the first step. If an exact line search is performed, then $\alpha_k = \arg \min J(\mathbf{x}_k + \alpha \mathbf{p}_k)$. In practice, an inexact line search usually suffices, with an acceptable $\alpha_k$ satisfying the Wolfe conditions.
3. Set $\mathbf{s}_k = \alpha_k \mathbf{p}_k$ and update $\mathbf{x}_{k+1} = \mathbf{x}_k + \mathbf{s}_k$.
4. Compute $\mathbf{y}_k = \nabla J(\mathbf{x}_{k+1}) - \nabla J(\mathbf{x}_k)$.
5. Update the approximate inverted Hessian matrix $H_{k+1} = H_k + \Delta H_k$, where:
   $$\Delta H_k = \frac{(\mathbf{s}_k^T \mathbf{y}_k + \mathbf{y}_k^T H_k \mathbf{y}_k)(\mathbf{s}_k \mathbf{s}_k^T)}{(\mathbf{s}_k^T \mathbf{y}_k)^2} - \frac{H_k \mathbf{y}_k \mathbf{s}_k^T + \mathbf{s}_k \mathbf{y}_k^T H_k}{\mathbf{s}_k^T \mathbf{y}_k} + \gamma I$$.
   
Here, $\Delta H_k$ is the correction term to update the Hessian approximation, $\mathbf{s}_k$ is the difference in parameter vectors, $\mathbf{y}_k$ is the difference in gradient vectors, $\gamma$ is a regularization parameter, and $I$ is the identity matrix.


In [7]:
import numpy as np
from scipy.optimize import minimize

def lbfgs_optimizer(objective_function, initial_parameters, max_iterations=100, tolerance=1e-6):
    def objective_wrapper(parameters):
        value, gradient = objective_function(parameters)
        return value, gradient

    optimized_result = minimize(
        objective_wrapper,
        initial_parameters,
        jac=True,
        method='L-BFGS-B',
        options={'maxiter': max_iterations, 'ftol': tolerance}
    )
    return optimized_result

# Define your objective function
def objective_function(parameters):
    x, y = parameters
    value = (x - 2) ** 2 + (y + 3) ** 2  # Example quadratic function
    gradient = np.array([2 * (x - 2), 2 * (y + 3)])  # Gradient of the function
    return value, gradient

# Example usage
initial_parameters = np.array([0.0, 0.0])  # Initial guess for parameters

# Use the L-BFGS optimizer to find the optimal parameters
optimized_result = lbfgs_optimizer(objective_function, initial_parameters)

# Print the results
print("Optimized parameters:", optimized_result.x)
print("Optimization status:", optimized_result.message)
